<a href="https://colab.research.google.com/github/jorcisai/APR/blob/master/neuralnets/src/mnist_local_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST loader

Importing standard packages and tensorflow_datasets to ease data manipulation.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf
import os

import tensorflow_datasets as tfds

First, you need to upload mnist_dataset.py and original MNIST data files from your drive

In [5]:
from google.colab import files

uploaded = files.upload()

Saving train-labels-idx1-ubyte.gz to train-labels-idx1-ubyte.gz
Saving train-images-idx3-ubyte.gz to train-images-idx3-ubyte.gz
Saving t10k-labels-idx1-ubyte.gz to t10k-labels-idx1-ubyte.gz
Saving t10k-images-idx3-ubyte.gz to t10k-images-idx3-ubyte.gz


Now, you are ready to load the MNIST data files and define train and dev sets

In [ ]:
from mnist_dataset import get_mnist

(train_images, train_labs), (test_images, test_labs) = get_mnist("./").load_data()
traindev_size = round(train_images.shape[0]*0.9)
dev_images = train_images[traindev_size:]
dev_labs = train_labs[traindev_size:]
train_images = train_images[:traindev_size]
train_labs = train_labs[:traindev_size]


Loading training and test images and labs from array into dataset object.

In [9]:
from mnist_dataset import get_mnist

(train_images, train_labs), (test_images, test_labs) = get_mnist("./").load_data()
train_images_dataset= tf.data.Dataset.from_tensor_slices(train_images)
train_labs_dataset= tf.data.Dataset.from_tensor_slices(train_labs)
test_images_dataset= tf.data.Dataset.from_tensor_slices(test_images)
test_labs_dataset= tf.data.Dataset.from_tensor_slices(test_labs)


Taking a look at a couple of training images and labels after being converted into dataset type.

In [ ]:
for train_image in train_images_dataset.take(1):
  print(train_image)

In [ ]:
for train_lab in train_labs_dataset.take(1):
  print(train_lab)

# Experimental design

Use tf.data.Dataset.take and tf.data.Dataset.skip to split training dataset into 90% for training and 10% for dev.

Before being passed into the model, the datasets need to be shuffled and batched. So, first, the complete dataset is shuffled with a fixed seed so that we can repeat the same shuffle of the dataset, then the dataset is split into training, validation and test, and each of these subsets is batched.

In [3]:
ls

sample_data/
